### Quickstart: Compare runs, choose a model, and deploy it to a REST API

In this quickstart, you will:

- Run a hyperparameter sweep on a training script

- Compare the results of the runs in the MLflow UI

- Choose the best run and register it as a model

- Deploy the model to a REST API

- Build a container image suitable for deployment to a cloud platform


In [17]:
import keras
import numpy as np
import pandas as pd
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import mlflow
from mlflow.models import infer_signature

In [18]:
## load the dataset
data=pd.read_csv(
    "https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv",
    sep=";",
)
data

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7


In [19]:
## Split the data into training,validation and test sets

train,test=train_test_split(data,test_size=0.25,random_state=42)
train

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
2835,6.3,0.25,0.22,3.30,0.048,41.0,161.0,0.99256,3.16,0.50,10.5,6
1157,7.8,0.30,0.29,16.85,0.054,23.0,135.0,0.99980,3.16,0.38,9.0,6
744,7.4,0.38,0.27,7.50,0.041,24.0,160.0,0.99535,3.17,0.43,10.0,5
1448,7.4,0.16,0.49,1.20,0.055,18.0,150.0,0.99170,3.23,0.47,11.2,6
3338,7.2,0.27,0.28,15.20,0.046,6.0,41.0,0.99665,3.17,0.39,10.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4426,6.2,0.21,0.52,6.50,0.047,28.0,123.0,0.99418,3.22,0.49,9.9,6
466,7.0,0.14,0.32,9.00,0.039,54.0,141.0,0.99560,3.22,0.43,9.4,6
3092,7.6,0.27,0.52,3.20,0.043,28.0,152.0,0.99129,3.02,0.53,11.4,6
3772,6.3,0.24,0.29,13.70,0.035,53.0,134.0,0.99567,3.17,0.38,10.6,6


In [20]:
train[['quality']].values.ravel()

array([6, 6, 5, ..., 6, 6, 8], shape=(3673,))

In [21]:
train_x=train.drop(['quality'],axis=1).values
train_y=train[['quality']].values.ravel()

## test dataset
test_x=test.drop(['quality'],axis=1).values
test_y=test[['quality']].values.ravel()

## splitting this train data into train and validation

train_x,valid_x,train_y,valid_y=train_test_split(train_x,train_y,test_size=0.20,random_state=42)

signature=infer_signature(train_x,train_y)

In [22]:
np.mean(train_x,axis=0)

array([6.86621852e+00, 2.80377808e-01, 3.32597005e-01, 6.42164738e+00,
       4.55513955e-02, 3.53556841e+01, 1.38792376e+02, 9.94074221e-01,
       3.18919333e+00, 4.88396869e-01, 1.05005673e+01])

In [23]:
train_x.shape[1]

11

In [24]:
### ANN Model

def train_model(params,epochs,train_x,train_y,valid_x,valid_y,test_x,test_y):

    ## Define model architecture
    mean=np.mean(train_x,axis=0)
    var=np.var(train_x,axis=0)

    model=keras.Sequential(
        [
            keras.Input([train_x.shape[1]]),
            keras.layers.Normalization(mean=mean,variance=var),
            keras.layers.Dense(64,activation='relu'),
            keras.layers.Dense(1)

        ]
    )

    ## compile the model
    model.compile(optimizer=keras.optimizers.SGD(
        learning_rate=params["lr"],momentum=params["momentum"]
    ),
    loss="mean_squared_error",
    metrics=[keras.metrics.RootMeanSquaredError()]
    )

    ## Train the ANN model with lr and momentum params wwith MLFLOW tracking
    with mlflow.start_run(nested=True):
        model.fit(train_x,train_y,validation_data=(valid_x,valid_y),
                  epochs=epochs,
                  batch_size=64)
        
        ## Evaluate the model
        eval_result=model.evaluate(valid_x,valid_y,batch_size=64)

        eval_rmse=eval_result[1]

        ## Log the parameters and results
        mlflow.log_params(params)
        mlflow.log_metric("eval_rmse",eval_rmse)

        ## log the model

        mlflow.tensorflow.log_model(model,"model",signature=signature)

        return {"loss": eval_rmse, "status": STATUS_OK, "model": model}

In [25]:
def objective(params):
    # MLflow will track the parameters and results for each run
    result = train_model(
        params,
        epochs=3,
        train_x=train_x,
        train_y=train_y,
        valid_x=valid_x,
        valid_y=valid_y,
        test_x=test_x,
        test_y=test_y,
    )
    return result

In [26]:
space={
    "lr":hp.loguniform("lr",np.log(1e-5),np.log(1e-1)),
    "momentum":hp.uniform("momentum",0.0,1.0)

}

In [27]:
mlflow.set_experiment("wine-quality")
with mlflow.start_run():
    # Conduct the hyperparameter search using Hyperopt
    trials=Trials()
    best=fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=4,
        trials=trials
    )

    # Fetch the details of the best run
    best_run = sorted(trials.results, key=lambda x: x["loss"])[0]

    # Log the best parameters, loss, and model
    mlflow.log_params(best)
    mlflow.log_metric("eval_rmse", best_run["loss"])
    mlflow.tensorflow.log_model(best_run["model"], "model", signature=signature)

    # Print out the best parameters and corresponding loss
    print(f"Best parameters: {best}")
    print(f"Best eval rmse: {best_run['loss']}")


Epoch 1/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 3:22 4s/step - loss: 39.8170 - root_mean_squared_error: 6.3101
 7/46 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 16.6693 - root_mean_squared_error: 3.9198 
31/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 6.9071 - root_mean_squared_error: 2.4291 
46/46 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - loss: 2.1506 - root_mean_squared_error: 1.4665 - val_loss: 0.7348 - val_root_mean_squared_error: 0.8572

Epoch 2/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step - loss: 0.5691 - root_mean_squared_error: 0.7544
34/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6478 - root_mean_squared_error: 0.8044 
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.6145 - root_mean_squared_error: 0.7839 - val_loss: 0.7240 - val_root_mean_squared_error: 0.8509

Epoch 3/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.7099 - root_mean_squared_error: 0.8425
25/46 

2025/11/30 12:45:33 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



Epoch 1/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 15s 341ms/step - loss: 40.1632 - root_mean_squared_error: 6.3374
25/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 39.4296 - root_mean_squared_error: 6.2792   
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 39.4188 - root_mean_squared_error: 6.2784 - val_loss: 39.4467 - val_root_mean_squared_error: 6.2807

Epoch 2/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - loss: 39.1511 - root_mean_squared_error: 6.2571
31/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 39.1577 - root_mean_squared_error: 6.2576 
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 38.7348 - root_mean_squared_error: 6.2237 - val_loss: 38.7566 - val_root_mean_squared_error: 6.2255

Epoch 3/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 38.8426 - root_mean_squared_error: 6.2324
41/46 ━

2025/11/30 12:46:54 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



Epoch 1/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 37s 833ms/step - loss: 33.9198 - root_mean_squared_error: 5.8241
20/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 36.2946 - root_mean_squared_error: 6.0242   
38/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 36.6228 - root_mean_squared_error: 6.0515
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 37.0665 - root_mean_squared_error: 6.0882 - val_loss: 36.6794 - val_root_mean_squared_error: 6.0564

Epoch 2/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - loss: 34.5662 - root_mean_squared_error: 5.8793
22/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 36.2981 - root_mean_squared_error: 6.0247 
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 35.9598 - root_mean_squared_error: 5.9966 - val_loss: 35.5833 - val_root_mean_squared_error: 5.9652

Epoch 3/3                                                                      

 1/46 ━

...

Epoch 1/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 24s 555ms/step - loss: 31.7236 - root_mean_squared_error: 5.6324
32/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 9.8984 - root_mean_squared_error: 2.9961    
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 3.3374 - root_mean_squared_error: 1.8268 - val_loss: 0.8814 - val_root_mean_squared_error: 0.9389

Epoch 2/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 2s 49ms/step - loss: 0.7533 - root_mean_squared_error: 0.8679
19/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.8404 - root_mean_squared_error: 0.9165 
37/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.8111 - root_mean_squared_error: 0.9003
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.7433 - root_mean_squared_error: 0.8621 - val_loss: 0.6576 - val_root_mean_squared_error: 0.8109

Epoch 3/3                                                                      

 1/46 ━━━━━━━━

2025/11/30 12:48:12 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



100%|██████████| 4/4 [03:16<00:00, 49.06s/trial, best loss: 0.7201569676399231]

2025/11/30 12:48:33 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



Best parameters: {'lr': np.float64(0.06993399541400645), 'momentum': np.float64(0.045659778386630046)}
Best eval rmse: 0.7201569676399231


In [28]:
## Inferencing

from mlflow.models import validate_serving_input

model_uri = 'runs:/c29658adbcb04fae83c21460d579a9ce/model'

# The logged model does not contain an input_example.
# Manually generate a serving payload to verify your model prior to deployment.
from mlflow.models import convert_input_example_to_serving_input

# Define INPUT_EXAMPLE via assignment with your own input example to the model
# A valid input example is a data instance suitable for pyfunc prediction
serving_payload = convert_input_example_to_serving_input(test_x)

# Validate the serving payload works on the model
validate_serving_input(model_uri, serving_payload)

d:\AIML\MLOPS\End-to-End MLOps Bootcamp\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step 


array([[-0.21243088],
       [ 0.01747451],
       [ 0.5809698 ],
       ...,
       [ 0.67971253],
       [ 0.47632194],
       [ 0.28373808]], shape=(1225, 1), dtype=float32)

In [29]:
# Load model as a PyFuncModel.
model_uri = 'runs:/c29658adbcb04fae83c21460d579a9ce/model'
loaded_model = mlflow.pyfunc.load_model(model_uri)

# Predict on a Pandas DataFrame.
import pandas as pd
loaded_model.predict(pd.DataFrame(test_x))

39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


array([[-0.21243088],
       [ 0.01747451],
       [ 0.5809698 ],
       ...,
       [ 0.67971253],
       [ 0.47632194],
       [ 0.28373808]], shape=(1225, 1), dtype=float32)

In [30]:
## Register in the model registry
mlflow.register_model(model_uri,"wine-quality")

d:\AIML\MLOPS\End-to-End MLOps Bootcamp\venv\lib\site-packages\mlflow\tracking\_model_registry\utils.py:215: FutureWarning: Filesystem model registry backend (e.g., './mlruns') is deprecated. Please switch to a database backend (e.g., 'sqlite:///mlflow.db'). For feedback, see: https://github.com/mlflow/mlflow/issues/18534
  return FileStore(store_uri)
Successfully registered model 'wine-quality'.
2025/11/30 12:53:25 WARNING mlflow.tracking._model_registry.fluent: Run with id c29658adbcb04fae83c21460d579a9ce has no artifacts at artifact path 'model', registering model based on models:/m-077ca81d426643fa9d4cca2801b32669 instead
Created version '1' of model 'wine-quality'.


<ModelVersion: aliases=[], creation_timestamp=1764487405802, current_stage='None', deployment_job_state=None, description=None, last_updated_timestamp=1764487405802, metrics=[<Metric: dataset_digest=None, dataset_name=None, key='eval_rmse', model_id='m-077ca81d426643fa9d4cca2801b32669', run_id='c29658adbcb04fae83c21460d579a9ce', step=0, timestamp=1764487053410, value=5.875644207000732>], model_id='m-077ca81d426643fa9d4cca2801b32669', name='wine-quality', params={'lr': '3.438500062042613e-05', 'momentum': '0.026839735321221192'}, run_id='c29658adbcb04fae83c21460d579a9ce', run_link=None, source='models:/m-077ca81d426643fa9d4cca2801b32669', status='READY', status_message=None, tags={}, user_id=None, version=1>